In [15]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.ensemble import StackingClassifier
import lightgbm as lgb
import sys
sys.path.append('../')
from utils.utils import *

In [16]:
X, y = get_training(exclude_obj=True)
X.dropna(inplace=True)
y = y.loc[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
def get_stacking():
    level0 = list()
    logreg = joblib.load('../models/model_bin/logistic_regression_model.pkl')
    params_lgbm = params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_leaves": 27,
    "learning_rate": 0.01,
    "min_child_samples": 50,
    "reg_alpha": 8.6234,
    "reg_lambda": 1.9843,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.99,
    'bagging_freq': 1, 
    "feature_pre_filter": False,               
    "boost_from_average": True,
    "num_class" : 5,
    "verbose": 1}
    lgbm = lgb.LGBMClassifier(**params_lgbm, callbacks=[lgb.early_stopping(100)], num_boost_round=10000)
    # lgbm = lgb.Booster(model_file='../models/model_bin/model5_FineTune_ObjDropped.txt')
    level0.append(('Logisitic Regression', logreg))
    level0.append(('Light Gradient Boosting', lgbm))
    level1 = LogisticRegression(multi_class='multinomial')
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=1)
    return model

In [18]:
stacking_clf = get_stacking()
stacking_clf.fit(X_train, y_train)


/Users/srijanchaudhuri/opt/miniconda3/envs/stat441/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/srijanchaudhuri/opt/miniconda3/envs/stat441/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/srijanchaudhuri/opt/miniconda3/envs/stat441/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Plea

TypeError: Cannot clone object '<lightgbm.basic.Booster object at 0x7fac30d31790>' (type <class 'lightgbm.basic.Booster'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [14]:
from sklearn.metrics import log_loss, accuracy_score

y_pred_proba = stacking_clf.predict_proba(X_test)
y_pred = stacking_clf.predict(X_test)

logloss = log_loss(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)

print(f"Log Loss: {logloss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

[LightGBM] [Warning] Unknown parameter: object
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: 0x7fac30d31af0>
[LightGBM] [Warning] Unknown parameter: callbacks
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.99, subsample=1.0 will be ignored. Current value: bagging_fraction=0.99
[LightGBM] [Warning] Unknown parameter: object
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: 0x7fabec1b10d0>
[LightGBM] [Warning] Unknown parameter: callbacks
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagg

In [7]:
stacking_clf.fit(X, y)

/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sc

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(cv=5,
                   estimators=[('Logisitic Regression',
                                LogisticRegression(multi_class='multinomial')),
                               ('Extreme Gradient Boosting',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_typ...
                                              n_estimators=None, n_jobs=None,
                                              num_class=5,
                                              num_parallel_tree=None, ...)),
                               ('Light Gradient Boosting',
                                LGBMClassifier(bagging_fraction=0.8,
                                               bagging_freq=5,
                                               boost_from_average=True,
                                               feature_fraction=0.8,
                                               learning_rate=0.01,
                                               metric='multi_logloss',
                                               n_estimators=2225, num_class=5,
                                               num_leaves=10,
                                               objective='multiclass'))],
                   final_estimator=LogisticRegression(multi_class='multinomial'))

In [28]:
# Load Test Dataset
X_submit = pd.read_csv('../data/X_test.csv', index_col=0)
X_submit = X_submit.select_dtypes(exclude=['object'])
#X_submit = X_submit.dropna(axis=1)
#predictions = stacking_clf.predict_proba(X)

In [13]:
preds_df = pd.DataFrame(predictions, columns=['no answer', 'very important', 'quite important', 'not important', 'not at all important'])
preds_df.index.name = 'id'
print(preds_df.shape)
preds_df.head()

(11437, 5)


,no answer,very important,quite important,not important,not at all important
id,,,,,
0,0.002688,0.824663,0.146116,0.016842,0.009691
1,0.008663,0.134339,0.757618,0.088446,0.010934
2,0.014567,0.091468,0.589843,0.264291,0.039832
3,0.014071,0.116724,0.715549,0.134676,0.018979
4,0.008724,0.226936,0.685516,0.066905,0.011918


In [14]:
preds_df.to_csv(f"../data/predictions/predictions4.csv")

In [26]:

X_submit = X_submit.dropna(axis=1)

In [29]:
predictions = stacking_clf.predict_proba(X_submit)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values